In [1]:
import requests
import time
import pandas as pd

In [2]:
url = 'https://www.reddit.com/r/Nike/hot/.json'

In [7]:
headers = {'User-agent': 'danhyunkim'}

In [8]:
res = requests.get(url, headers=headers)

In [9]:
res.status_code

200

In [11]:
nike_json = res.json()

In [16]:
nike_json.keys()

dict_keys(['kind', 'data'])

In [19]:
sorted(nike_json['data'].keys())

['after', 'before', 'children', 'dist', 'modhash']

In [28]:
len(nike_json['data']['children'])

26

In [59]:
nike_df = pd.DataFrame(nike_json['data']['children'])
nike_df

,data,kind
0,"{'approved_at_utc': None, 'subreddit': 'Nike',...",t3
1,"{'approved_at_utc': None, 'subreddit': 'Nike',...",t3
2,"{'approved_at_utc': None, 'subreddit': 'Nike',...",t3
3,"{'approved_at_utc': None, 'subreddit': 'Nike',...",t3
4,"{'approved_at_utc': None, 'subreddit': 'Nike',...",t3
5,"{'approved_at_utc': None, 'subreddit': 'Nike',...",t3
6,"{'approved_at_utc': None, 'subreddit': 'Nike',...",t3
7,"{'approved_at_utc': None, 'subreddit': 'Nike',...",t3
8,"{'approved_at_utc': None, 'subreddit': 'Nike',...",t3
9,"{'approved_at_utc': None, 'subreddit': 'Nike',...",t3


We need the title of the thread, the subreddit the post belongs to, the length of time the post has been up, and the number of comments on the thread

Probably going to have to set up a for loop to go through all the posts

In [84]:
nike_json['data']['children'][0]['data']

{'approved_at_utc': None,
 'subreddit': 'Nike',
 'selftext': 'Please note that any product from a previous season may be hard to locate. You may be limited to paying collectors type prices on online auction sites. Beware that many products sold on suspect 3rd party sites are counterfeit product.\n\n**Shoes:**\n\nIf you have access to the shoe, look behind the tongue or inside the shoe for the size tag. There is a 6 digit code followed by a hyphen and 3 digits representing the model and color. Search Google for "Nike" followed by the code and you\'ll likely find the correct shoe.\n',
 'author_fullname': 't2_4azwb',
 'saved': False,
 'mod_reason_title': None,
 'gilded': 0,
 'clicked': False,
 'title': 'Tips for identifying Nike Product',
 'link_flair_richtext': [],
 'subreddit_name_prefixed': 'r/Nike',
 'hidden': False,
 'pwls': None,
 'link_flair_css_class': 'grey',
 'downs': 0,
 'thumbnail_height': None,
 'hide_score': False,
 'name': 't3_47fex4',
 'quarantine': False,
 'link_flair_tex

t3_a5sszn
t3_a5sszn


[None, None]

Get the name of the last post (nike_json['data']['after']) 

Use that name and to hit the following url to get more posts from the next page: https://www.reddit.com/r/Nike/hot/.json?after=THE_AFTER_NAME

Create a loop to repeat the first 2 steps

In [32]:
param = {'after':'t3_a5sszn'}

In [33]:
requests.get(url, params=param, headers=headers)

<Response [200]>

In [43]:
nike_json['data']['after']

't3_a5sszn'

In [48]:
posts = []
after = None
for i in range(2):
    print(i)
    if after == None:
        params = {}
    else: 
        params = {'after': after}
    url = 'https://www.reddit.com/r/Nike/hot/.json'
    res = requests.get(url, params=params, headers=headers)
    if res.status_code == 200:
        the_json = res.json()
        posts.extend(the_json['data']['children'])
        after = the_json['data']['after']
    else:
        print(res.status_code)
        break
    time.sleep(5)

0
1


In [82]:
df = pd.DataFrame(posts)
df.head()

,data,kind
0,"{'approved_at_utc': None, 'subreddit': 'Nike',...",t3
1,"{'approved_at_utc': None, 'subreddit': 'Nike',...",t3
2,"{'approved_at_utc': None, 'subreddit': 'Nike',...",t3
3,"{'approved_at_utc': None, 'subreddit': 'Nike',...",t3
4,"{'approved_at_utc': None, 'subreddit': 'Nike',...",t3


In [112]:
posts[50]['data']['title']

'what Are Your Thoughts On The Nike Air Fear Of God 1'

In [113]:
posts[50]['data']['subreddit_subscribers']

11001

In [115]:
posts[50]['data']['created_utc']

1544431138.0

In [98]:
range(len(posts))

range(0, 51)

In [117]:
nike_cells = []
for i in range(len(posts)):
    nike_cells.append({'title': posts[i]['data']['title'], 'subreddit': posts[i]['data']['subreddit'], 
                      'comments': posts[i]['data']['num_comments'], 'text': posts[i]['data']['selftext'], 
                      'subscribers': posts[i]['data']['subreddit_subscribers'], 'url': posts[i]['data']['url'],
                      'created': posts[i]['data']['created_utc']})
    
nike_df = pd.DataFrame(nike_cells)

nike_df

,comments,created,subreddit,subscribers,text,title,url
0,1,1.456352e+09,Nike,11001,Please note that any product from a previous s...,Tips for identifying Nike Product,https://www.reddit.com/r/Nike/comments/47fex4/...
1,2,1.544983e+09,Nike,11001,,Nike Airmax 270 Volt,https://i.redd.it/rpcgvf3gio421.jpg
2,7,1.544981e+09,Nike,11001,,Saw these at the shops today. Never seen them ...,https://i.redd.it/d1rhxiwsao421.jpg
3,1,1.545001e+09,Nike,11001,,Ben Simmons' Nike Hyperdunk x PE,https://imgur.com/w4QWxEK
4,12,1.544955e+09,Nike,11001,,Got these for my bday. Anyone knows ehat color...,https://i.redd.it/hj3l2pbk7m421.jpg
5,0,1.544995e+09,Nike,11001,,Can someone ID this vintage Nike hoodie?,https://imgur.com/gallery/yDNvQfK
6,0,1.544990e+09,Nike,11001,,look at this rad shoe I made,https://store.nike.com/nl/nl_nl/product/sb-ste...
7,2,1.544965e+09,Nike,11001,123,123,https://www.reddit.com/r/Nike/comments/a6otd1/...
8,8,1.544892e+09,Nike,11001,"From a pure tech standpoint, what’s the differ...",AF1 vs Air Max 1 vs AJ1?,https://www.reddit.com/r/Nike/comments/a6gm4j/...
9,1,1.544941e+09,Nike,11001,&amp;#x200B;\n\n&amp;#x200B;\n\n&amp;#x200B;\n...,Full Nike utilitarian set up,https://www.reddit.com/r/Nike/comments/a6mv9w/...
